In [1]:
import pandas as pd
import numpy as np
import os


os.getcwd()

'c:\\Users\\fabri\\OneDrive\\Documentos\\Estudos\\ANAC'

# Functions

In [4]:
def read_airfares():
    archives = os.listdir("Raw")

    final_df = []
    

    for count in archives:
        local_df = pd.read_csv(f'Raw/{count}',sep=';',encoding = 'ISO-8859-1',usecols=lambda column: column not in ['Unnamed: 0'])
        if local_df.shape[1] == 1:
            local_df = pd.read_csv(f'Raw/{count}',sep=',',encoding = 'ISO-8859-1',usecols=lambda column: column not in ['Unnamed: 0'])

        local_df = pd.DataFrame(local_df)
        local_df = local_df.set_axis(['ANO','MES','EMPRESA','ORIGEM','DESTINO','TARIFA','ASSENTOS'],axis=1)    

        if count == archives[0]:
            final_df = local_df
        else:
            final_df = pd.concat([final_df,local_df])
    return final_df


def read_auxiliary():

    public_airports = pd.read_csv("Auxiliar/AerodromosPublicos.csv",
                              encoding = 'ISO-8859-1',
                              header = 1,
                              sep=';'
                              )
    
    companies = pd.read_csv("Auxiliar/pda_empresas_aereas_nacionais.csv",sep=';',header=1)

    companies = companies[companies['ICAO'].notna()]

    return public_airports,companies


def merging_auxiliary(airfares,public_airports,companies):
    
    df_merged = pd.merge(airfares,
            companies[['ICAO','Razao','Servico','Estrangeira']],
            how='left',
            left_on = 'EMPRESA',
            right_on = 'ICAO')

    origin = public_airports[['Código OACI','Município','UF']]
    origin = origin.set_axis(['ORIGEM','origin_city','origin_state'],axis=1)

    df_merged = pd.merge(df_merged,
                            origin,
                            how='left',
                            on = 'ORIGEM')

    destiny = public_airports[['Código OACI','Município','UF']]
    destiny = origin.set_axis(['DESTINO','destiny_city','destiny_state'],axis=1)

    df_merged = pd.merge(df_merged,
                            destiny,
                            how='left',
                            on = 'DESTINO')
    
    return df_merged




## Bloco 1 - Carga de dado 
#### Adição de colunas auxiliares como deparas e criação de arquivo de trabalho

In [3]:
airfares = read_airfares()

public_airports,companies = read_auxiliary()

In [47]:
airfares


,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA,ASSENTOS
0,2021,1,AZU,SBAC,SBBR,"293,20",1
1,2021,1,AZU,SBAC,SBBR,"340,90",6
2,2021,1,AZU,SBAC,SBBR,"367,30",1
3,2021,1,AZU,SBAC,SBBR,"435,90",1
4,2021,1,AZU,SBAC,SBBR,"452,90",1
...,...,...,...,...,...,...,...
762001,2024,11,TAM,SWLC,SBCT,1367.86,2
762002,2024,11,TAM,SWLC,SBCT,2288.56,1
762003,2024,11,TAM,SWLC,SBGL,2778.11,5
762004,2024,11,TAM,SWLC,SBGL,2855.38,2


In [ ]:
public_airports



,Código OACI,CIAD,Nome,Município,UF,Município Servido,UF Servido,LATGEOPOINT,LONGEOPOINT,Latitude,...,Superfície 1,Designação 2,Comprimento 2,Largura 2,Resistência 2,Superfície 2,Situação,Validade do Registro,Portaria de Registro,Link Portaria
0,SDZG,CE0008,Pedro Teixeira Castelo Regional Tauá,TAUÁ,Ceará,Tauá,Ceará,-59.333.333,-402.975,"05°56'00""S",...,Asfalto,NaN,NaN,NaN,NaN,NaN,Cadastrado,07/05/2030,PA2020-1189,https://pergamum.anac.gov.br/arquivos/PA2020-1...
1,SDIG,SP0038,Ibitinga,IBITINGA,São Paulo,Ibitinga,São Paulo,-21.747.222,-48.855.833,"21°44'50""S",...,Asfalto,NaN,NaN,NaN,NaN,NaN,Cadastrado,23/04/2030,PA2020-1074,https://pergamum.anac.gov.br/arquivos/PA2020-1...
2,SBVT,ES0001,Eurico de Aguiar Salles,VITÓRIA,Espírito Santo,Vitória,Espírito Santo,-20.258.056,-40.286.389,"20°15'29""S",...,Asfalto,fev/20,2058.0,45,86/F/B/X/T,Asfalto,Cadastrado,NaN,PA2024-13.827,https://pergamum.anac.gov.br/arquivos/PA2024-1...
3,SNJK,BA0047,Jequié,JEQUIÉ,Bahia,Jequié,Bahia,-13.877.778,-40.071.389,"13°52'40""S",...,Asfalto,NaN,NaN,NaN,NaN,NaN,Cadastrado,18/11/2024,PA2014-2695,https://pergamum.anac.gov.br/arquivos/PA2014-2...
4,SSKU,SC0016,Lauro Antônio da Costa,CURITIBANOS,Santa Catarina,Curitibanos,Santa Catarina,-27.287.222,-50.603.889,"27°17'14""S",...,Asfalto,NaN,NaN,NaN,NaN,NaN,Cadastrado,20/05/2030,PA2020-1296,https://pergamum.anac.gov.br/arquivos/PA2020-1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,SNKF,MG0071,Das Bandeirinhas,CONSELHEIRO LAFAIETE,Minas Gerais,Conselheiro Lafaiete,Minas Gerais,-20.738.611,-437.975,"20°44'19""S",...,Asfalto,NaN,NaN,NaN,NaN,NaN,Cadastrado,16/10/2028,PA2022-8286,https://pergamum.anac.gov.br/arquivos/PA2022-8...
493,SSRS,MA0008,Barreirinhas,BARREIRINHAS,Maranhão,Barreirinhas,Maranhão,-27.566.667,-42.803.333,"02°45'24""S",...,Asfalto,NaN,NaN,NaN,NaN,NaN,Cadastrado,28/06/2033,PA2023-11665,https://pergamum.anac.gov.br/arquivos/PA2023-1...
494,SWWA,GO0012,Porangatu,PORANGATU,Goiás,Porangatu,Goiás,-13.404.444,-49.157.222,"13°24'16""S",...,Asfalto,NaN,NaN,NaN,NaN,NaN,Cadastrado,19/05/2030,PA2020-1293,https://pergamum.anac.gov.br/arquivos/PA2020-1...
495,SWKT,GO0015,Catalão,CATALÃO,Goiás,Catalão,Goiás,-18.216.944,-47.899.444,"18°13'01""S",...,Asfalto,NaN,NaN,NaN,NaN,NaN,Cadastrado,21/09/2030,PA2020-2421,https://pergamum.anac.gov.br/arquivos/PA2020-2...


In [ ]:

merged_airfares = merging_auxiliary(airfares=airfares,public_airports=public_airports,companies=companies)
merged_airfares





,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA,ASSENTOS,ICAO,Razao,Servico,Estrangeira,origin_city,origin_state,destiny_city,destiny_state
0,2021,1,AZU,SBAC,SBBR,"293,20",1,AZU,AZUL LINHAS AÉREAS BRASILEIRAS S/A,"TRANSPORTE AÉREO NÃO REGULAR, TRANSPORTE AÉREO...",AD,ARACATI,Ceará,BRASÍLIA,Distrito Federal
1,2021,1,AZU,SBAC,SBBR,"340,90",6,AZU,AZUL LINHAS AÉREAS BRASILEIRAS S/A,"TRANSPORTE AÉREO NÃO REGULAR, TRANSPORTE AÉREO...",AD,ARACATI,Ceará,BRASÍLIA,Distrito Federal
2,2021,1,AZU,SBAC,SBBR,"367,30",1,AZU,AZUL LINHAS AÉREAS BRASILEIRAS S/A,"TRANSPORTE AÉREO NÃO REGULAR, TRANSPORTE AÉREO...",AD,ARACATI,Ceará,BRASÍLIA,Distrito Federal
3,2021,1,AZU,SBAC,SBBR,"435,90",1,AZU,AZUL LINHAS AÉREAS BRASILEIRAS S/A,"TRANSPORTE AÉREO NÃO REGULAR, TRANSPORTE AÉREO...",AD,ARACATI,Ceará,BRASÍLIA,Distrito Federal
4,2021,1,AZU,SBAC,SBBR,"452,90",1,AZU,AZUL LINHAS AÉREAS BRASILEIRAS S/A,"TRANSPORTE AÉREO NÃO REGULAR, TRANSPORTE AÉREO...",AD,ARACATI,Ceará,BRASÍLIA,Distrito Federal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21685352,2024,11,TAM,SWLC,SBCT,1367.86,2,TAM,TAM LINHAS AÉREAS S.A.,TRANSPORTE AÉREO REGULAR,JJ,RIO VERDE,Goiás,CURITIBA,Paraná
21685353,2024,11,TAM,SWLC,SBCT,2288.56,1,TAM,TAM LINHAS AÉREAS S.A.,TRANSPORTE AÉREO REGULAR,JJ,RIO VERDE,Goiás,CURITIBA,Paraná
21685354,2024,11,TAM,SWLC,SBGL,2778.11,5,TAM,TAM LINHAS AÉREAS S.A.,TRANSPORTE AÉREO REGULAR,JJ,RIO VERDE,Goiás,RIO DE JANEIRO,Rio de Janeiro
21685355,2024,11,TAM,SWLC,SBGL,2855.38,2,TAM,TAM LINHAS AÉREAS S.A.,TRANSPORTE AÉREO REGULAR,JJ,RIO VERDE,Goiás,RIO DE JANEIRO,Rio de Janeiro


In [6]:
# merged_airfares.to_csv('Processado/Tarifas Carregadas 2021 a 2024.csv')
